In [1]:
import torch

# Force GPU since you confirmed it's available
device = torch.device("cuda")
print("Using device:", torch.cuda.get_device_name(0))

Using device: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
import os, glob, random
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from tqdm import tqdm

In [3]:
def make_stratified_splits(root_dir, out_dir, test_size=0.10, val_size=0.10, seed=42):
    os.makedirs(out_dir, exist_ok=True)
    items, labels= [], []
    classes= sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
    for cls in classes:
        pattern= os.path.join(root_dir, cls, '*')
        for p in glob.glob(pattern):
            if p.lower().endswith(('.jpg', '.png', '.jpeg')):
                items.append(p)
                labels.append(cls)
    df=pd.DataFrame({'path': items , 'label': labels})
    temp_size= test_size+val_size
    train_df, temp_df= train_test_split(df, test_size= temp_size, stratify= df['label'], random_state=seed)
    test_rel=test_size/temp_size
    val_df, test_df= train_test_split(temp_df, test_size=test_rel, stratify= temp_df['label'], random_state=seed)
    train_df.to_csv(os.path.join(out_dir, 'train.csv'), index=False)
    val_df.to_csv(os.path.join(out_dir, 'val.csv'), index=False)
    test_df.to_csv(os.path.join(out_dir, 'test.csv'), index=False)
    print(f"Splits saved to {out_dir} (train/val/test sizes: {len(train_df)}/{len(val_df)}/{len(test_df)})")
    return sorted(list(df['label'].unique()))

In [4]:
class PlantDataset(Dataset):
    def __init__(self, csv_file, class_to_idx=None, transform=None):
        self.df=pd.read_csv(csv_file)
        self.paths=self.df['path'].tolist()
        self.labels=self.df['label'].tolist()
        if class_to_idx is None:
            cls_unique= sorted(list(set(self.labels)))
            self.class_to_idx = {c:i for i,c in enumerate(cls_unique)}
        else:
            self.class_to_idx= class_to_idx
        self.targets= [self.class_to_idx[l] for l in self.labels]
        self.transform= transform

    def __len__(self):
        return len(self.paths)


    def __getitem__(self, idx):
        img= Image.open(self.paths[idx]).convert('RGB')
        if self.transform:
            img= self.transform(img)
        label= self.targets[idx]
        return img, label

In [5]:
#transform
train_tf= transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

val_tf= transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

if __name__ == "__main__":
    ROOT= "C:\\Users\\warma\\Downloads\\archive\\PlantVillage"
    OUT= "splits"
    classes=make_stratified_splits(ROOT, OUT)
    class_to_idx= {c:i for i,c in enumerate(classes)}
    train_ds = PlantDataset(os.path.join(OUT,'train.csv'), class_to_idx=class_to_idx, transform=train_tf)
    val_ds   = PlantDataset(os.path.join(OUT,'val.csv'),   class_to_idx=class_to_idx, transform=val_tf)
    test_ds  = PlantDataset(os.path.join(OUT,'test.csv'),  class_to_idx=class_to_idx, transform=val_tf)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
    val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)
    test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

    print("Done. Classes:", classes)
    

Splits saved to splits (train/val/test sizes: 16510/2064/2064)
Done. Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class LeafNet(nn.Module):
    def __init__(self, num_classes=38):
        super(LeafNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(64)

        self.pool  = nn.MaxPool2d(2, 2)

        
        self.fc1   = nn.Linear(64 * 28 * 28, 256)
        self.drop  = nn.Dropout(0.3)
        self.fc2   = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)

        # classifier
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x


In [7]:
import torch.optim as optim
device= "cuda" if torch.cuda.is_available() else "cpu"
num_classes= len(classes)
model = LeafNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()

optimizer= optim.AdamW(model.parameters(), lr= 1e-3, weight_decay= 1e-4)
scheduler= optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=3, factor=0.5)


In [8]:
from tqdm import tqdm
def train_model(model, train_dl, val_dl, criterion, optimizer, scheduler, epochs=20):
    device = torch.device("cuda")
    model = model.to(device)
    best_val_acc= 0.0
    history= {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    for epoch in range(epochs):
        model.train()
        train_loss, correct, total= 0.0, 0, 0
        
        for xb, yb in tqdm(train_dl, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
            optimizer.zero_grad()

            outputs= model(xb)
            loss= criterion(outputs, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*xb.size(0)
            _, preds= torch.max(outputs, 1)
            correct += (preds==yb).sum().item()
            total += yb.size(0)

        train_loss /= total
        train_acc= correct/total



        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for xb, yb in tqdm(val_dl, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                xb, yb = xb.to(device), yb.to(device)
                outputs = model(xb)
                loss = criterion(outputs, yb)

                val_loss += loss.item() * xb.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)

        val_loss /= total
        val_acc = correct / total


        print(f"Epoch {epoch+1}/{epochs}: "
              f"Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, "
              f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        scheduler.step(val_loss)
        if val_acc > best_val_acc:
            best_val_acc= val_acc
            torch.save(model.state_dict(), "best_LeafNet.pth")

    return history


In [12]:
history= train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=20)

Epoch 1/20 [Val]: 100%|██████████| 65/65 [00:44<00:00,  1.45it/s]


Epoch 1/20: Train Loss=2.5673, Train Acc=0.3052, Val Loss=1.6763, Val Acc=0.4477


Epoch 2/20 [Val]: 100%|██████████| 65/65 [00:09<00:00,  6.55it/s]


Epoch 2/20: Train Loss=1.7160, Train Acc=0.4277, Val Loss=1.3714, Val Acc=0.5402


Epoch 3/20 [Val]: 100%|██████████| 65/65 [00:05<00:00, 11.98it/s]


...

Epoch 18/20: Train Loss=0.9890, Train Acc=0.6541, Val Loss=0.7941, Val Acc=0.7766


Epoch 19/20 [Val]: 100%|██████████| 65/65 [00:05<00:00, 10.84it/s]


Epoch 19/20: Train Loss=0.9857, Train Acc=0.6537, Val Loss=0.6029, Val Acc=0.8096


Epoch 20/20 [Val]: 100%|██████████| 65/65 [00:05<00:00, 11.35it/s]

Epoch 20/20: Train Loss=0.9692, Train Acc=0.6566, Val Loss=0.7132, Val Acc=0.7926


In [ ]:

model.load_state_dict(torch.load("best_LeafNet.pth"))
model.eval()


correct, total = 0, 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        outputs = model(xb)
        _, preds = torch.max(outputs, 1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)
test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")

C:\Users\warma\AppData\Local\Temp\ipykernel_11404\711082015.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_LeafNet.pth"))


Test Accuracy: 0.8028
